<a href="https://colab.research.google.com/github/Sneha-mv23/PetVision/blob/main/dataset_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download Stanford Dogs image dataset
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar


In [ ]:
# Create a folder and extract images.tar
!mkdir images
!tar -xf images.tar -C images


In [ ]:
import os, shutil, random

def split_dataset(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.15):
    os.makedirs(dest_dir, exist_ok=True)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dest_dir, split), exist_ok=True)

    for breed in os.listdir(source_dir):
        breed_path = os.path.join(source_dir, breed)
        if not os.path.isdir(breed_path):
            continue
        images = os.listdir(breed_path)
        random.shuffle(images)

        train_end = int(len(images) * train_ratio)
        val_end = int(len(images) * (train_ratio + val_ratio))

        for i, img in enumerate(images):
            if i < train_end:
                split = 'train'
            elif i < val_end:
                split = 'val'
            else:
                split = 'test'

            dest_path = os.path.join(dest_dir, split, breed)
            os.makedirs(dest_path, exist_ok=True)

            shutil.copy(os.path.join(breed_path, img), os.path.join(dest_path, img))

# Call the function to split the dataset
split_dataset("images/Images", "data")


In [ ]:
import os

for split in ['train', 'val', 'test']:
    path = os.path.join("data", split)
    total_images = sum([len(files) for r, d, files in os.walk(path)])
    print(f"{split} images: {total_images}")


In [ ]:
 import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
image_size = 224  # Standard input size for CNNs like ResNet, VGG

# For training: Resize, RandomCrop, Flip, Normalize
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# For validation and test: Resize and Normalize only
val_test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# Define paths
train_path = 'data/train'
val_path = 'data/val'
test_path = 'data/test'

# Load datasets with transforms
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_path, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(root=test_path, transform=val_test_transforms)

batch_size = 32  # You can try 64 or 128 later

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# Helper function to unnormalize and show image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get one batch from train loader
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show 1st 8 images in the batch
imshow(torchvision.utils.make_grid(images[:8]))




In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms

class PetVisionCNN(nn.Module):
    def __init__(self, num_classes):
        super(PetVisionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # (B, 3, 224, 224) -> (B, 32, 224, 224)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # (B, 32, 112, 112)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # (B, 64, 56, 56)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                          # (B, 128, 28, 28)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                     # Flatten to (B, 128*28*28)
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    # Number of classes = number of dog breeds
num_classes = len(train_dataset.classes)

# Instantiate the model
model = PetVisionCNN(num_classes)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward + Backward + Optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f"Validation Accuracy: {val_accuracy:.2f}%")

torch.save(model.state_dict(), 'petvision_model.pth')